## DQN_project_MVA
### Caroline Favart

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [ ]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        
        
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [15]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function act returns the next action that will be performed. The action is a number of the set of possible actions (whose size is given by n_action).  
>
> epsilon is essential as it corresponds to the probability to perform a random action versus to perform a learned action. It is the exploration rate.
>
> Performing random actions will be the core of the training, especially at the beginning, as it will allow the algorithm to explore the possibilities and learn which actions increase the reward. Thus, epsilon needs to be high at the beginning.
>
> On the other side, when we perform a learned action, we are exploiting what we know by performing actions increasing the reward. If we have not explored enough before, this will not correspond to the actions which will maximize the possible reward. But at some point, it is important to perform exploitation to increase the total reward. We could decrease epsilon with time, to exploit what has been learned.
>
> The choice of epsilon highlights the tradeoff between exploration and exploitation.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [16]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        
        # add a "contour" for the sea
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        # 3rd dimension corresponds to the color RGB
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        
        # position of the rat:
        b[self.x,self.y,:]=256
        
        # black for the contours :
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        # Board represents where there are cheesy or poisonous cells
        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [21]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging
epochs_test=30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The array *position* represents the game cells, with zeros everywhere except in its contours (representing the sea, where we have -1) and at the rat's position (indicated by 1). We reinitialize the *position* array at each round, thus we only have the current rat's position and not the previous ones. 
>
> The array *board* represents the board of the game : all the cells of the island and the sea around, with indicated the poisonous cells and the cells with cheese. Indeed, the cells contain a value corresponding to the malus or bonus that will be added to the reward if the rat moves to it. Once the rat moves to a cell, its value becomes zero. The positions of rewards and poison are randomly (with a binomial) set at each reset of the game, just like the rat's initial position (with random.randint).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [40]:
# the random agent performs a random action in the set of possible actions

# we know that the rat has 4 possible actions : up, left, down, right
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """ 
        Act via the policy of the agent, from a given state s (position(x,y) of the rat at time before)
        it proposes an action a"""
        a=np.random.randint(0,self.n_action,size=1)[0]
        
        return a
        pass 

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [26]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        print(e)
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [27]:
# Initialize the game

env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random4.mp4'))

0
Win/lose count 7.0/18.0. Average score (-11.0)
1
Win/lose count 7.5/20.0. Average score (-11.75)
2
Win/lose count 6.0/7.0. Average score (-8.166666666666666)
3
Win/lose count 5.0/9.0. Average score (-7.125)
4
Win/lose count 8.5/12.0. Average score (-6.4)
Final score: -6.4


> As we could have forecasted, the average score obtained with only random actions is very low. 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




>We know that:
>
>\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\geq 0}^\infty \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> = E_{p^{\pi}}[r(s_0,a_0)+\sum_{t\geq 1}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
>\end{equation*}
>By changing the sum index, we obtain : 
>\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)+\gamma\sum_{t\geq 0}^\infty\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \end{equation*}
>This is equivalent to :
>\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)]+\gamma E_{p^{\pi}}[\sum_{t\geq 0}^\infty\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]]= E_{(s',a')\sim p(.|s,a)}[r(s,a)]+E_{(s',a')\sim p(.|s,a)}[\gamma E_{p^{\pi}}[\sum_{t\geq 0}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']] \end{equation*} by decaying the first index through a changement of variables to a' and s'.
>Thus, we obtain :
>\begin{equation*}Q^\pi(s,a)= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma E_{p^{\pi}}[\sum_{t\geq 0}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'] \end{equation*}
>and finally : 
>\begin{equation*}Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
>\end{equation*}


  
> Then, we assume its existence and consider the optimal policy $\pi^*$. $Q^{*}$ is the optimal value function, obtained with the optimal policy. We can write :   
>
>\begin{equation*}
Q^{*}(s,a)=\max_{\pi}Q^{\pi}(s,a) \end{equation*}
>
>Using the above equation, we obtain :
>
>\begin{equation*} Q^{*}(s,a)=\max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] = \max_{\pi}E_{(s',a')\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{*}(s',a')]\end{equation*}
>Maximizing the second part of the equation on $\pi$ is equivalent to maximize on a'. Therefore, we obtain : 
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}


  
> Thus, we will consider as objective function the loss between the optimal state value function Q computed above and the state value function given by our model. Using the norm, we obtain :
> \begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [64]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # First case : the memory is not full.
        # We just need to append the new move to the memory list.
        if len(self.memory) < self.max_memory : 
            self.memory.append(m)
            
        # Second case : the memory is full.
        # We need to replace one element (randomly chosen) of the memory with the new one.
        else :
            self.memory[np.random.randint(len(self.memory))]=m
        pass

    def random_access(self):
        # We return a random element of memory.
        index=np.random.randint(len(self.memory))
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [65]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [125]:
from keras.layers import Dropout, Flatten

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        
        # We compute the model's prediction for rewards for each possible action from state s
        # As s is an only one dimensional array, we need to use np.expand_dims to turn it into a two dimensional one and use it in the model. 
        prediction = self.model.predict(np.expand_dims(s, 0))[0]
        
        # The learned_act corresponds to the action yielding the highest prediction :
        return np.argmax(prediction)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        
        # Two steps: first memorize the states, second learn from the pool
        
        # Memorize the states : 
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
                
        # We want for each batch to store each state element (s_, n_s_, a_, r_, game_over_) and the reward for each possible actions
        
        # Initialization of the arrays : 
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
             # Sample from the memory :
            state, next_state, action, reward, game_over = self.memory.random_access()
            # Fill in the input array :
            input_states[i]=state.copy()
            # Initializing the output
            output_q=[0,0,0,0]

            # If game_over, this is the last state so we only compute the corresponding reward :
            if game_over_:
                output_q[action] = reward
            
            # If it is not game_over, our rat has a view over two cells, thus, we also add the reward of the next step (max predicted reward)
            else :
                output_q[action] = reward + self.discount * max(self.model.predict(np.expand_dims(next_state, 0))[0])
            # We store our Q-value in target_q
            target_q[i, :] = output_q.copy()
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        
        # Computing the loss with train_on_batch function : 
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        #######

        # Initialization :
        model=Sequential()
        
        # We flattent the data :
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        
        # We will use  three fully connected layers : 
        model.add(Dense(32, activation='relu' ))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(4, activation='linear'))

        # Compiling the model :
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        
        self.model = model
        

In [126]:
epochs_train=100
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fccc_train')
# HTML(display_videos('fc_train10.mp4'))

Epoch 000/100 | Loss 0.0261 | Win/lose count 9.5/9.0 (0.5)
Epoch 001/100 | Loss 0.0262 | Win/lose count 9.0/8.0 (1.0)
Epoch 002/100 | Loss 0.0233 | Win/lose count 3.0/7.0 (-4.0)
Epoch 003/100 | Loss 0.0309 | Win/lose count 4.5/6.0 (-1.5)
Epoch 004/100 | Loss 0.0061 | Win/lose count 9.0/10.0 (-1.0)
Epoch 005/100 | Loss 0.0205 | Win/lose count 6.5/8.0 (-1.5)
Epoch 006/100 | Loss 0.0095 | Win/lose count 3.5/5.0 (-1.5)
Epoch 007/100 | Loss 0.0122 | Win/lose count 5.5/9.0 (-3.5)
Epoch 008/100 | Loss 0.0082 | Win/lose count 7.0/9.0 (-2.0)
Epoch 009/100 | Loss 0.0034 | Win/lose count 4.0/5.0 (-1.0)
Epoch 010/100 | Loss 0.0112 | Win/lose count 4.5/3.0 (1.5)
Epoch 011/100 | Loss 0.0192 | Win/lose count 10.0/6.0 (4.0)
Epoch 012/100 | Loss 0.0049 | Win/lose count 5.5/5.0 (0.5)
Epoch 013/100 | Loss 0.0086 | Win/lose count 2.5/2.0 (0.5)
Epoch 014/100 | Loss 0.0058 | Win/lose count 2.5/5.0 (-2.5)
Epoch 015/100 | Loss 0.0031 | Win/lose count 3.5/6.0 (-2.5)
Epoch 016/100 | Loss 0.0032 | Win/lose count

KeyboardInterrupt: 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [74]:
# We will now use a CNN for the DQN algorithm :

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # First model tried : two convolutional layers (64 and 32) and one Dense(4)
        model=Sequential()
        model.add(Conv2D(64, (3,3), activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(32, (3,3), activation='relu'))
        
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model


In [75]:
# Training the model :
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0085 | Win/lose count 3.5/3.0 (0.5)
Epoch 001/100 | Loss 0.0038 | Win/lose count 6.0/6.0 (0.0)
Epoch 002/100 | Loss 0.0225 | Win/lose count 13.0/4.0 (9.0)
Epoch 003/100 | Loss 0.0043 | Win/lose count 4.5/3.0 (1.5)
Epoch 004/100 | Loss 0.0041 | Win/lose count 5.5/5.0 (0.5)
Epoch 005/100 | Loss 0.0040 | Win/lose count 10.0/5.0 (5.0)
Epoch 006/100 | Loss 0.0119 | Win/lose count 5.5/1.0 (4.5)
Epoch 007/100 | Loss 0.0101 | Win/lose count 8.0/8.0 (0.0)
Epoch 008/100 | Loss 0.0051 | Win/lose count 3.0/1.0 (2.0)
Epoch 009/100 | Loss 0.0064 | Win/lose count 6.0/3.0 (3.0)
Epoch 010/100 | Loss 0.0127 | Win/lose count 5.5/7.0 (-1.5)
Epoch 011/100 | Loss 0.0128 | Win/lose count 11.5/5.0 (6.5)
Epoch 012/100 | Loss 0.0048 | Win/lose count 10.5/4.0 (6.5)
Epoch 013/100 | Loss 0.0116 | Win/lose count 13.0/4.0 (9.0)
Epoch 014/100 | Loss 0.0039 | Win/lose count 11.0/3.0 (8.0)
Epoch 015/100 | Loss 0.0188 | Win/lose count 6.5/3.0 (3.5)
Epoch 016/100 | Loss 0.0104 | Win/lose count 18.0

In [77]:
# Changing the size of the convolutional layers : 
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model=Sequential()
        model.add(Conv2D(16, (3,3), activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(8, (3,3), activation='relu'))
        
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [78]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.25, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnnn_train')

Epoch 000/100 | Loss 0.0182 | Win/lose count 9.0/5.0 (4.0)
Epoch 001/100 | Loss 0.0137 | Win/lose count 5.0/10.0 (-5.0)
Epoch 002/100 | Loss 0.0097 | Win/lose count 8.0/4.0 (4.0)
Epoch 003/100 | Loss 0.0119 | Win/lose count 9.5/14.0 (-4.5)
Epoch 004/100 | Loss 0.0311 | Win/lose count 9.0/8.0 (1.0)
Epoch 005/100 | Loss 0.0123 | Win/lose count 6.5/10.0 (-3.5)
Epoch 006/100 | Loss 0.0038 | Win/lose count 10.0/6.0 (4.0)
Epoch 007/100 | Loss 0.0096 | Win/lose count 4.5/9.0 (-4.5)
Epoch 008/100 | Loss 0.0103 | Win/lose count 12.0/4.0 (8.0)
Epoch 009/100 | Loss 0.0095 | Win/lose count 2.5/6.0 (-3.5)
Epoch 010/100 | Loss 0.0034 | Win/lose count 4.0/5.0 (-1.0)
Epoch 011/100 | Loss 0.0203 | Win/lose count 9.0/3.0 (6.0)
Epoch 012/100 | Loss 0.0175 | Win/lose count 5.5/4.0 (1.5)
Epoch 013/100 | Loss 0.0077 | Win/lose count 6.5/2.0 (4.5)
Epoch 014/100 | Loss 0.0119 | Win/lose count 11.0/2.0 (9.0)
Epoch 015/100 | Loss 0.0053 | Win/lose count 9.0/7.0 (2.0)
Epoch 016/100 | Loss 0.0146 | Win/lose count

In [93]:
# BEST MODEL : two convolutional layers and one FC one 
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model=Sequential()
        model.add(Conv2D(32, (3,3), activation='relu', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(16, (3,3), activation='relu'))

        # model.add(MaxPooling2D(pool_size=3))
        
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [94]:
# Training 
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.25, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnnnn_train')

Epoch 000/100 | Loss 0.0130 | Win/lose count 6.0/8.0 (-2.0)
Epoch 001/100 | Loss 0.0075 | Win/lose count 5.5/3.0 (2.5)
Epoch 002/100 | Loss 0.0050 | Win/lose count 9.0/12.0 (-3.0)
Epoch 003/100 | Loss 0.0061 | Win/lose count 8.0/12.0 (-4.0)
Epoch 004/100 | Loss 0.0251 | Win/lose count 8.5/6.0 (2.5)
Epoch 005/100 | Loss 0.0196 | Win/lose count 17.5/6.0 (11.5)
Epoch 006/100 | Loss 0.0126 | Win/lose count 8.0/4.0 (4.0)
Epoch 007/100 | Loss 0.0111 | Win/lose count 14.5/7.0 (7.5)
Epoch 008/100 | Loss 0.0100 | Win/lose count 18.0/8.0 (10.0)
Epoch 009/100 | Loss 0.0251 | Win/lose count 15.0/9.0 (6.0)
Epoch 010/100 | Loss 0.0073 | Win/lose count 8.0/5.0 (3.0)
Epoch 011/100 | Loss 0.0116 | Win/lose count 9.0/6.0 (3.0)
Epoch 012/100 | Loss 0.0202 | Win/lose count 16.0/8.0 (8.0)
Epoch 013/100 | Loss 0.0159 | Win/lose count 11.5/9.0 (2.5)
Epoch 014/100 | Loss 0.0232 | Win/lose count 13.5/11.0 (2.5)
Epoch 015/100 | Loss 0.0057 | Win/lose count 17.0/12.0 (5.0)
Epoch 016/100 | Loss 0.0046 | Win/lose 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [121]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
agent_cnn.load(name_weights='cnnnn_trainmodel.h5',name_model='cnnnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
agent_cnn.load(name_weights='fcc_trainmodel.h5',name_model='fcc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnnnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fcc_test')

Test of the CNN
0
Win/lose count 5.0/3.0. Average score (2.0)
1
Win/lose count 9.0/8.0. Average score (1.5)
2
Win/lose count 3.5/1.0. Average score (1.8333333333333333)
3
Win/lose count 2.0/1.0. Average score (1.625)
4
Win/lose count 4.5/3.0. Average score (1.6)
5
Win/lose count 4.0/5.0. Average score (1.1666666666666667)
6
Win/lose count 14.5/4.0. Average score (2.5)
7
Win/lose count 3.5/2.0. Average score (2.375)
8
Win/lose count 5.0/2.0. Average score (2.4444444444444446)
9
Win/lose count 10.0/3.0. Average score (2.9)
10
Win/lose count 4.0/1.0. Average score (2.909090909090909)
11
Win/lose count 5.5/4.0. Average score (2.7916666666666665)
12
Win/lose count 8.0/4.0. Average score (2.8846153846153846)
13
Win/lose count 5.0/4.0. Average score (2.75)
14
Win/lose count 6.5/4.0. Average score (2.7333333333333334)
15
Win/lose count 4.5/4.0. Average score (2.59375)
16
Win/lose count 7.5/8.0. Average score (2.411764705882353)
17
Win/lose count 8.5/4.0. Average score (2.5277777777777777)
18
W

In [ ]:
HTML(display_videos('cnn_test10.mp4'))

In [ ]:
HTML(display_videos('fc_test10.mp4'))

> We can observe that the CNN performs poorly in the test, in comparison to the training.
> It still performs better than the FC algorithm, which does not seem to converge.
> By observing the videos, we can see that the main issues are the fact that not all the board is explored and the tendency in testing to reproduce the same moves again and again.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



I first reloaded the Agent and CNN to adjust n_state.

In [101]:
from keras.layers import Dropout, Flatten

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=3):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        prediction = self.model.predict(np.expand_dims(s, 0))[0]
        return np.argmax(prediction)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
                
        # for each batch, store each state element (s_, n_s_, a_, r_, game_over_)
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        
        # store the reward for each possible actions
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
             # Sample from the memory :
            state, next_state, action, reward, game_over = self.memory.random_access()
            # Fill in the arrays :
            input_states[i]=state.copy()
            # Predicting the model output for this state : (we need to modify the shape)
            output_q=[0,0,0,0]

            # If game_over, this is the last state so we only compute the corresponding reward :
            if game_over_:
                output_q[action] = reward
            
            # If it is not game_over, our rate has a view over two cells, thus, we also add the reward of 
            else :
                output_q[action] = reward + self.discount * max(self.model.predict(np.expand_dims(next_state, 0))[0])
            
            target_q[i, :] = output_q.copy()
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        #######

        model=Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        #model.add(Dense(32, activation='relu' ))
        # model.add(Dropout(0.2))
        model.add(Dense(16, activation='relu'))
        # model.add(Dropout(0.2))
        model.add(Dense(4, activation='linear'))

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        
        self.model = model
        

In [107]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model=Sequential()
        model.add(Conv2D(32, (3,3), activation='relu', input_shape=(5, 5, 3)))
        model.add(Conv2D(16, (3,3), activation='relu'))

        # model.add(MaxPooling2D(pool_size=3))
        
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

Then, I defined the new functions :

In [115]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            
            # Add the train=true argument
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')    
        
        pass
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        
        # add a "contour" for the sea
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        # 3rd dimension for the color RGB
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        
        # where the rat is :
        b[self.x,self.y,:]=256
        
        # black for the sea
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        # Adding a malus (only if train==True) when the rat visits a cell already visited : 
        reward=0
        if train:
            reward = -self.malus_position[self.x, self.y]
        
        reward = reward + self.board[self.x, self.y]
        
        # Updating the malus_position array : 
        self.malus_position[self.x, self.y] = 0.01
        
        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2 : 
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        # Board represents where there are cheesy or poisonous cells
        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        # 3 features :
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state    


In [116]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 100, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0086 | Win/lose count 8.0/11.619999999999965 (-3.6199999999999655)
Epoch 001/100 | Loss 0.0203 | Win/lose count 13.91/6.549999999999973 (7.360000000000027)
Epoch 002/100 | Loss 0.0164 | Win/lose count 5.930000000000001/4.849999999999983 (1.0800000000000178)
Epoch 003/100 | Loss 0.0124 | Win/lose count 7.920000000000002/6.789999999999963 (1.130000000000039)
Epoch 004/100 | Loss 0.0083 | Win/lose count 3.45/2.9299999999999806 (0.5200000000000196)
Epoch 005/100 | Loss 0.0033 | Win/lose count 5.420000000000001/2.879999999999983 (2.540000000000018)
Epoch 006/100 | Loss 0.0098 | Win/lose count 6.3900000000000015/3.8599999999999826 (2.530000000000019)
Epoch 007/100 | Loss 0.0056 | Win/lose count 10.320000000000002/1.7600000000000013 (8.56)
Epoch 008/100 | Loss 0.0047 | Win/lose count 12.370000000000003/5.6399999999999855 (6.730000000000017)
Epoch 009/100 | Loss 0.0202 | Win/lose count 3.920000000000001/5.929999999999969 (-2.009999999999968)
Epoch 010/100 | Loss 0.0034 |

Epoch 081/100 | Loss 0.0087 | Win/lose count 21.559999999999988/5.569999999999978 (15.990000000000009)
Epoch 082/100 | Loss 0.0072 | Win/lose count 14.700000000000005/5.709999999999966 (8.990000000000038)
Epoch 083/100 | Loss 0.0121 | Win/lose count 14.210000000000004/4.7099999999999795 (9.500000000000025)
Epoch 084/100 | Loss 0.0148 | Win/lose count 11.270000000000003/5.779999999999966 (5.4900000000000375)
Epoch 085/100 | Loss 0.0113 | Win/lose count 13.720000000000004/8.729999999999965 (4.990000000000039)
Epoch 086/100 | Loss 0.0154 | Win/lose count 5.3900000000000015/7.899999999999964 (-2.5099999999999625)
Epoch 087/100 | Loss 0.0189 | Win/lose count 7.350000000000002/3.8599999999999826 (3.4900000000000198)
Epoch 088/100 | Loss 0.0081 | Win/lose count 17.64/8.62999999999998 (9.010000000000021)
Epoch 089/100 | Loss 0.0113 | Win/lose count 20.089999999999993/7.59999999999997 (12.490000000000023)
Epoch 090/100 | Loss 0.0168 | Win/lose count 17.64/8.649999999999967 (8.990000000000034)
E

In [118]:
# Evaluation
epochs_test=30
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

0
Win/lose count 13.720000000000004/5.729999999999971. Average score (7.990000000000033)
1
Win/lose count 21.06999999999999/9.579999999999979. Average score (9.740000000000023)
2
Win/lose count 19.599999999999994/5.60999999999998. Average score (11.156666666666688)
3
Win/lose count 5.3900000000000015/4.899999999999961. Average score (8.490000000000025)
4
Win/lose count 12.250000000000004/4.759999999999966. Average score (8.29000000000003)
5
Win/lose count 25.479999999999976/4.48999999999998. Average score (10.40666666666669)
6
Win/lose count 4.410000000000001/2.919999999999981. Average score (9.132857142857166)
7
Win/lose count 13.230000000000004/5.739999999999979. Average score (8.927500000000023)
8
Win/lose count 16.170000000000005/6.679999999999981. Average score (8.990000000000023)
9
Win/lose count 13.230000000000004/4.739999999999976. Average score (8.940000000000023)
10
Win/lose count 24.49999999999998/5.5099999999999705. Average score (9.853636363636385)
11
Win/lose count 20.579

> This is the best performing algorithm. Focusing more on exploration allows the rat to learn which actions are increasing its reward and yields better results.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***